In [1]:
# control random processes
import torch
torch.manual_seed(1)
import numpy as np
np.random.seed(1)

In [2]:
from network import NeuralNetwork

In [3]:
network = NeuralNetwork()

In [4]:
network.train(game_type='mini_iterative', game_penalty_type='scores', random_games=True, test=True)

0.1389 0.0454 81


In [ ]:
import pandas as pd
# mini-iterative, random_games=True, 5 hours
before = pd.DataFrame([[ 16,   0],
 [  32,   32],
 [  64,  351],
 [ 128, 1020],
 [ 256,  556],
 [ 512,   41]])



after = pd.DataFrame([[ 16,   1],
 [ 32,  20],
 [ 64, 236],
 [128, 758],
 [256, 910],
 [512,  75]])

import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(2,1, sharex=True, sharey=True)

ax1.bar(before[0].astype(str), before[1])
ax1.set_ylabel('Before')

ax2.bar(after[0].astype(str), after[1])
ax2.set_ylabel('After')

fig.suptitle('Number of games reaching tile value,\nbefore and after training')

fig.show()

In [ ]:
import matplotlib.pyplot as plt

plt.plot([x for x in network.model.scores[256] if x!=0])
plt.show()

In [ ]:
params = {'game_type':['full', 'mini_iterative', 'mini_random'],
         'random_games':[False, True],
         'random_frac':[None, np.random.uniform(0, 1)],
         'batch_size':10,
          'lr':0.001,
         'move_penalty_type':[None,'nonzero','linear_move_num','exponential_move_num'],
         'game_penalty_type':[None,'scores','max','log2_max','tile_sums']}

In [ ]:
# test parameter combinations

for game_type in params['game_type']:
    for random_games in params['random_games']:
        for random_frac in params['random_frac']:
            for move_penalty_type in params['move_penalty_type']:
                for game_penalty_type in params['game_penalty_type']:
                    network.train(test=True, duration=1/60000,
                                 game_type=game_type, random_games=random_games, random_frac=random_frac,
                                 move_penalty_type=move_penalty_type, game_penalty_type=game_penalty_type)